##### The cell below is for you to keep track of the libraries used and install those libraries quickly
##### Ensure that the proper library names are used and the syntax of `%pip install PACKAGE_NAME` is followed

In [2]:
%pip install pandas 
%pip install matplotlib
%pip install scikit-learn
# add commented pip installation lines for packages used as shown above for ease of testing
# the line should be of the format %pip install PACKAGE_NAME 


Note: you may need to restart the kernel to use updated packages.



## **DO NOT CHANGE** the filepath variable
##### Instead, create a folder named 'data' in your current working directory and 
##### have the .parquet file inside that. A relative path *must* be used when loading data into pandas

In [2]:
# Can have as many cells as you want for code
import pandas as pd
import os
os.getcwd()
filepath = "./data/catB_train.parquet" 
# the initialised filepath MUST be a relative path to a folder named data that contains the parquet file

### **ALL** Code for machine learning and dataset analysis should be entered below. 
##### Ensure that your code is clear and readable.
##### Comments and Markdown notes are advised to direct attention to pieces of code you deem useful.

In [3]:
###...code...###
# pd.show_versions()
df = pd.read_parquet(filepath)
df.head() # can only use 710 values to 
# list(dat)
df['f_purchase_lh'] = df['f_purchase_lh'].fillna(0)
# corr_matrix = df.corr()["f_purchase_lh"] #returns the correlation matrix between each column and "f_purchase_lh"
df['f_purchase_lh']
# corr_matrix.sort_values(ascending=False).head(n=10)

19550    0.0
4600     0.0
13337    0.0
15074    0.0
19724    0.0
        ... 
11284    0.0
11964    0.0
5390     0.0
860      0.0
15795    0.0
Name: f_purchase_lh, Length: 17992, dtype: float64

At first glance, let's use the client risk indicators as our features.

In [4]:
# X = df[features]
# print(X.head())
# for f in features:
#     print(f)
#     print(df[df[f].isna()])

# df_substandard = df[df['flg_substandard'].isna()]
# df_borderline = df[df['flg_is_borderline_standard'].isna()]
# print('sbustandard = borderline?: ', df_substandard.equals(df_borderline))

# df_revised = df[df['flg_is_revised_term'].isna()]
# print('revised term = substandard?: ', df_revised.equals(df_substandard))
# print('revised term = borderline?: ',df_revised.equals(df_borderline))

# df_rental = df[df['flg_is_rental_flat'].isna()]
# print('rental flat = substandard?: ', df_rental.equals(df_substandard))
# print('rental flat = borderline?: ', df_rental.equals(df_borderline))
# print('rental flat = revised?: ', df_rental.equals(df_revised))

# y = df['f_purchase_lh']


# from sklearn.model_selection import train_test_split
# train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
# from sklearn.neighbors import KNeighborsClassifier
# neigh = KNeighborsClassifier(n_neighbors=5)
# neigh.fit(train_X, train_y)

features = ['flg_substandard', 'flg_is_borderline_standard', 'flg_is_revised_term', 'flg_is_rental_flat', 
            'flg_has_health_claim', 'flg_has_life_claim', 'flg_gi_claim', 'flg_is_proposal', 'flg_with_preauthorisation', 
            'flg_is_returned_mail']

flag = True
for f in features:
    for g in features:
        if f == g:
            continue
        df_f = df[df[f].isna()]
        df_g = df[df[g].isna()]
        if not df_f.equals(df_g):
            flag = False

print(flag)

True


Now we have a problem.  Our flag indicators have three possible values: 1, 0 and NaN.
Thankfully, after evaluation we see that all the ones with NaN in the flag variables are the same, so should we throw them away?

On the other hand, shall we interpolate it? or use KNN imputation?

Nevermind, are there any entries where no flag variables are keyed in, but they bought the policy?

In [23]:
# Since they're all NaN, let's just pick one column where it's NaN and then condense from there

df[(df['flg_substandard'].isna()) & (df['f_purchase_lh'] == 0)] # 10 rows where the flag is gone but they purchased, out of 17992
# 1004 rows have NaN in flag, but didn't purchase
# 1014 total NaN in flag


# df
# df[df[features].isna() & df['f_purchase_lh'] == 1]

,clntnum,race_desc,ctrycode_desc,clttype,stat_flag,min_occ_date,cltdob_fix,cltsex_fix,flg_substandard,flg_is_borderline_standard,...,recency_giclaim,giclaim_cnt_success,recency_giclaim_success,giclaim_cnt_unsuccess,recency_giclaim_unsuccess,flg_gi_claim_29d435_ever,flg_gi_claim_058815_ever,flg_gi_claim_42e115_ever,flg_gi_claim_856320_ever,f_purchase_lh
18743,eae4336879,None,Singapore,G,ACTIVE,2016-12-01,1982-06-04,Female,NaN,NaN,...,NaN,None,None,None,None,None,None,None,None,0.0
7319,d305956170,Chinese,Not Applicable,P,ACTIVE,2022-07-01,1965-01-19,Male,NaN,NaN,...,NaN,None,None,None,None,None,None,None,None,0.0
7930,d8fd7b908e,Chinese,Singapore,P,ACTIVE,2007-06-01,1984-12-16,Female,NaN,NaN,...,NaN,None,None,None,None,None,None,None,None,0.0
11378,3962e4e3b3,Indian,Singapore,P,ACTIVE,2022-09-05,1973-06-08,Female,NaN,NaN,...,NaN,None,None,None,None,None,None,None,None,0.0
5224,f62a986db1,Chinese,Singapore,P,ACTIVE,2010-05-31,1970-05-18,Female,NaN,NaN,...,NaN,None,None,None,None,None,None,None,None,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7989,6c4d06d974,None,Singapore,G,ACTIVE,2022-08-01,1985-05-18,Female,NaN,NaN,...,NaN,None,None,None,None,None,None,None,None,0.0
10233,7a452c429c,None,Singapore,G,ACTIVE,2022-07-01,1961-12-19,Male,NaN,NaN,...,NaN,None,None,None,None,None,None,None,None,0.0
189,2bc01b439c,None,Singapore,G,ACTIVE,2022-09-01,1990-04-20,Female,NaN,NaN,...,NaN,None,None,None,None,None,None,None,None,0.0
18431,00f2beb549,Chinese,Singapore,P,ACTIVE,2011-03-31,1971-03-08,Female,NaN,NaN,...,NaN,None,None,None,None,None,None,None,None,0.0


So, investigating the correlation matrix, we find that the columns that are correlated to `f_purchase_lh` are as follows:
1. `f_ever_bought_gi`, has the customer ever bought general insurance?
2. `flg_gi_claim`, a flag value to determine whether the customer has claimed a general insurance policy.


I am using this block to find all columns with 'ape_' in it.

In [27]:
ape_cols = [data for data in df if 'ape' in data]
ape_cols
df[['flg_substandard', 'f_purchase_lh']]


,flg_substandard,f_purchase_lh
19550,0.0,NaN
4600,0.0,NaN
13337,0.0,NaN
15074,0.0,NaN
19724,0.0,NaN
...,...,...
11284,0.0,NaN
11964,0.0,NaN
5390,0.0,NaN
860,0.0,NaN


clttype = P, C, G?
We hypothesise that P means personal, C is company, G is general!!!

In [19]:
non_null = df[~df['f_purchase_lh'].isnull()] # never buy
ones_only = df[df['f_purchase_lh'] == 1.0] # got buy

True

In [5]:
df[df['clttype'] == 'C']

,clntnum,race_desc,ctrycode_desc,clttype,stat_flag,min_occ_date,cltdob_fix,cltsex_fix,flg_substandard,flg_is_borderline_standard,...,recency_giclaim,giclaim_cnt_success,recency_giclaim_success,giclaim_cnt_unsuccess,recency_giclaim_unsuccess,flg_gi_claim_29d435_ever,flg_gi_claim_058815_ever,flg_gi_claim_42e115_ever,flg_gi_claim_856320_ever,f_purchase_lh
5472,be9efce8b5,None,Singapore,C,ACTIVE,2017-08-31,None,None,0.0,0.0,...,0.0,None,None,None,None,None,None,None,None,NaN
19271,ef11214f4b,None,Singapore,C,ACTIVE,2023-03-07,None,None,0.0,0.0,...,NaN,None,None,None,None,None,None,None,None,NaN
14274,503d5036d7,None,Singapore,C,LAPSED,2022-12-15,None,None,0.0,0.0,...,NaN,None,None,None,None,None,None,None,None,NaN
3881,50c6f06299,None,Singapore,C,ACTIVE,2019-05-13,None,None,0.0,0.0,...,NaN,None,None,None,None,None,None,None,None,NaN
8988,126f48686a,None,Singapore,C,ACTIVE,2020-07-14,None,None,0.0,0.0,...,NaN,None,None,None,None,None,None,None,None,NaN
3219,08aa0f66c9,None,Singapore,C,ACTIVE,2016-07-22,None,None,0.0,0.0,...,48.0,None,None,None,None,None,None,None,None,NaN
19184,e7c8c72720,None,Singapore,C,ACTIVE,2013-07-01,None,None,0.0,0.0,...,4.0,None,None,None,None,None,None,None,None,1.0
4541,ef9a34930a,None,Singapore,C,ACTIVE,2020-12-29,2006-06-14,None,0.0,0.0,...,NaN,None,None,None,None,None,None,None,None,NaN
5795,6b87430f80,None,Singapore,C,ACTIVE,2013-04-19,None,None,0.0,0.0,...,NaN,None,None,None,None,None,None,None,None,NaN
5003,0fba75d376,None,Singapore,C,LAPSED,2023-03-18,None,None,0.0,0.0,...,NaN,None,None,None,None,None,None,None,None,NaN


## The cell below is **NOT** to be removed
##### The function is to be amended so that it accepts the given input (dataframe) and returns the required output (list). 
##### It is recommended to test the function out prior to submission
-------------------------------------------------------------------------------------------------------------------------------
##### The hidden_data parsed into the function below will have the same layout columns wise as the dataset *SENT* to you
##### Thus, ensure that steps taken to modify the initial dataset to fit into the model are also carried out in the function below

In [14]:
def testing_hidden_data(hidden_data: pd.DataFrame) -> list:
    '''DO NOT REMOVE THIS FUNCTION.

The function accepts a dataframe as input and return an iterable (list)
of binary classes as output.

The function should be coded to test on hidden data
and should include any preprocessing functions needed for your model to perform. 
    
All relevant code MUST be included in this function.'''
    result = [] 
    return result

##### Cell to check testing_hidden_data function

In [ ]:
# This cell should output a list of predictions.
test_df = pd.read_parquet(filepath)
test_df = test_df.drop(columns=["f_purchase_lh"])
print(testing_hidden_data(test_df))

### Please have the filename renamed and ensure that it can be run with the requirements above being met. All the best!